In [11]:
%%capture
import pandas as pd
import numpy as np
import os
import re
import gensim
import nltk
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from nltk import tokenize    
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = data.text.count()
items = []

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    
    patterns = [word_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words

def build_sentences(dataset):
    sentences = []
    
    for entry in dataset.title:            
        title = []
        for ngram in parse(entry):
            title.append(ngram)
        
        sentences.append(title)
    return sentences

sents = build_sentences(data)